# **Portfolio Selection Assignment**

In [1]:
# Imports
import pandas as pd
import numpy as np

In [ ]:
# Function to allocate portfolio
def allocate_portfolio(tickers, last_price, weights, total_portfolio_value=100000):
    """
    Allocate portfolio based on weights, scale to portfolio value, 
    and compute integer share allocations.
    
    Parameters
    ----------
    tickers : list of str
        List of stock tickers.
    last_price : np.ndarray
        Array of last prices for each stock.
    weights : np.ndarray
        Array of portfolio weights for each stock.
    total_portfolio_value : float, optional
        Total value of the portfolio to allocate (default is 100000).
    
    Returns
    -------
    results : pd.DataFrame
        DataFrame with ticker, final shares, and leftover cash info.
    """
    
    # Scaling the ratios to the total portfolio value
    dollar_allocation = weights * total_portfolio_value

    # Calculating the number of shares to buy for each stock
    pre_round_shares = dollar_allocation / last_price

    # Rounding shares
    # Because of the strict "no-over-allocation" rule, rounding down should be used
    # But if the generally preferred simple rounding does not exceed the the total allocation it should be used
    if (np.round(pre_round_shares) * last_price).sum() > total_portfolio_value:
        final_shares = np.floor(pre_round_shares).astype(int)
    else:
        final_shares = np.round(pre_round_shares).astype(int)

    # Calculating used capital and leftover cash
    actual_total_investment = sum(final_shares * last_price)

    # Display results
    print(f"Actual total investment in risky assets: ${actual_total_investment:.0f}")
    print(f"Leftover cash to invest in risky-free asset: ${(total_portfolio_value - actual_total_investment):.0f}")

    # Only the necessary columns
    results = pd.DataFrame({
        "Ticker": tickers,
        "Final Shares": final_shares
    })

    return results

## **Part I.**
### **Week 1 - Classic Positive Momentum Strategy**

In [4]:
# Loading data
prices_w1 = pd.read_excel("Week 1/week_1_price_data.xlsx")

# Calculating the 12-month return
# Only if last year's closing price is not zero
prices_w1["return_12m"] = np.where(prices_w1["Close_2022"] != 0,
                                   prices_w1["Close_2023"] / prices_w1["Close_2022"] - 1,
                                   0)

# Filtering for positive returns
prices_w1["positive_return"] = np.where(prices_w1["return_12m"] > 0,
                                        prices_w1["return_12m"],
                                        0)

# Calculating the weight for each positive return stock
prices_w1["portfolio_weight"] = prices_w1["positive_return"] / sum(prices_w1["positive_return"])

# Allocating portfolio
results = allocate_portfolio(
    tickers=prices_w1["Ticker_symbol"],
    last_price=prices_w1["Close_2023"],
    weights=prices_w1["portfolio_weight"])

# Exporting results
results.to_excel("Week 1/week_1_results.xlsx")

Actual total investment in risky assets: $95454
Leftover cash to invest in risky-free asset: $4546


## **Part II.**

### **Week 2 - Tangency Portfolio using Mean-Variance Analysis**

In [22]:
# Loading data
prices_w2 = pd.read_excel("Week 2/week_2_price_data.xlsx")

# Take absolute price values
# Some prices are negative due to data errors
prices_w2["Price"] = prices_w2["Price"].abs()

# Drop rows missing essential fields to avoid unexpected NaNs
prices_w2 = prices_w2.dropna(subset=["Date", "Ticker Symbol", "Price"])

# Transforming the data to wide format
prices_w2_wide = prices_w2.pivot_table(index="Date", columns="Ticker Symbol", values="Price", aggfunc="last")

# Sort the index (dates) to ensure chronological order
prices_w2_wide = prices_w2_wide.sort_index()

# Drop columns with any NaN values to ensure clean return calculations
# And filter out stocks that might not longer be traded
prices_w2_wide = prices_w2_wide.dropna(axis=1)

# Compute returns
returns = prices_w2_wide.pct_change()

# Drop the first row
returns = returns.drop(returns.index[0])

# Expected returns (mean per period)
mu = returns.mean().values

# Covariance matrix
Sigma = returns.cov().values

# Stock tickers
tickers = returns.columns

# Inverse covariance
Sigma_inv = np.linalg.inv(Sigma)

# Tangency weights (without risk-free rate)
raw_weights = Sigma_inv @ mu
#tangency_weights = raw_weights / np.sum(raw_weights)

# Set negative weights to zero and renormalize
raw_weights = np.maximum(raw_weights, 0)
tangency_weights = raw_weights / raw_weights.sum() if raw_weights.sum() > 0 else raw_weights

# Allocating portfolio
results = allocate_portfolio(
    tickers=tickers,
    last_price=prices_w2_wide.iloc[-1].values,
    weights=tangency_weights)

# Exporting results
results.to_excel("Week 2/week_2_results.xlsx")

Actual total investment in risky assets: $95710
Leftover cash to invest in risky-free asset: $4290


### **Week 3 - ?**

### **Week 4 - ?**

### **Week 5 - ?**

### **Week 6 - ?**